In [1]:
import os
import sys
os.chdir('/home/peitian_zhang/Codes/News-Recommendation')
sys.path.append('/home/peitian_zhang/Codes/News-Recommendation')

import torch
from utils.utils import evaluate,train,prepare,load_hparams
from models.NPA import NPAModel
from configs.ManualConfig import hparams

## setting up the *NPA* model

### define paths and hyperparameters, load data

all these hyper parameters are fixed according to the paper [\[23\] Npa Neural news recommendation with personalized attention](https://dl.acm.org/doi/abs/10.1145/3292500.3330665)

- *mode*: data to read (*demo*/*small*/*large*)

- *batch_size*: size of each minibatch

- *title_size*: max word capacity of title

- *his_size*: max record capacity of click history

- *npratio*: number of negtive sampling

- *dropout_p*: probability of dropout layer

- *filter_num*: number of kernels in 1D CNN, which is also embedding dimension of news/user

- *embedding_dim*: word embedding dimension

- *user_dim*: user id embedding dimension

- *preference_dim*: user preference embedding dimension

In [2]:
hparams['filter_num'] = 400

In [3]:
vocab, loaders = prepare(hparams)

[2021-03-21 13:23:15,656] INFO (root) Hyper Parameters are
{'name': 'npa', 'epochs': 1, 'scale': 'demo', 'mode': 'train', 'batch_size': 5, 'title_size': 20, 'his_size': 50, 'npratio': 4, 'dropout_p': 0.2, 'filter_num': 400, 'embedding_dim': 300, 'preference_dim': 200, 'user_dim': 50, 'metrics': 'auc,mean_mrr,ndcg@5,ndcg@10', 'attrs': ['title'], 'device': 'cuda:0', 'k': -1, 'save_step': [0]}
[2021-03-21 13:23:15,658] INFO (root) preparing dataset...
[2021-03-21 13:23:18,097] INFO (torchtext.vocab) Loading vectors from .vector_cache/glove.840B.300d.txt.pt


In [4]:
npaModel = NPAModel(vocab=vocab,hparams=hparams,user_num=len(loaders[0].dataset.uid2index)).to(hparams['device'])

In [6]:
train(npaModel, hparams, loaders, interval=10)

[2021-03-21 09:53:20,871] INFO (root) training...
epoch 1 , step 590 , loss: 1.5587: : 593it [00:21, 27.04it/s]
[2021-03-21 09:53:43,555] INFO (root) saved model of epoch 1 at data/model_params/npa/demo_epoch1_step0_[hs=50,topk=-1].model
[2021-03-21 09:53:43,621] INFO (root) evaluating...
3625it [00:44, 81.10it/s]
[2021-03-21 09:54:29,340] INFO (root) evaluation results:{'auc': 0.5564, 'mean_mrr': 0.2502, 'ndcg@5': 0.2662, 'ndcg@10': 0.3367, 'epoch': 1, 'step': 0}


NPAModel(
  (encoder): NPA_Encoder(
    (embedding): Embedding(54076, 300)
    (softmax): Softmax(dim=-1)
    (user_embedding): Embedding(2426, 50)
    (wordPrefProject): Linear(in_features=50, out_features=200, bias=True)
    (wordQueryProject): Linear(in_features=200, out_features=400, bias=True)
    (CNN): Conv1d(300, 400, kernel_size=(3,), stride=(1,), padding=(1,))
    (RELU): ReLU()
    (Tanh): Tanh()
    (DropOut): Dropout(p=0.2, inplace=False)
  )
  (user_embedding): Embedding(2426, 50)
  (newsPrefProject): Linear(in_features=50, out_features=200, bias=True)
  (newsQueryProject): Linear(in_features=200, out_features=400, bias=True)
  (RELU): ReLU()
  (softmax): Softmax(dim=-1)
  (Tanh): Tanh()
  (DropOut): Dropout(p=0.2, inplace=False)
)